In [51]:
#-*- coding:utf-8 -*-
#Edited by bighead 19-1-20
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [52]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    with tf.name_scope('weights'):
        Weights = tf.Variable(initial)
        return Weights

In [53]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    with tf.name_scope('bias'):
        bias = tf.Variable(initial)
        return bias

In [54]:
def con2d(x, W):
    with tf.name_scope("con2d"):
        con2d_layer = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
        return con2d_layer

In [55]:
def pooling(x):
    with tf.name_scope('pooling'):
        pooling_layer = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        return pooling_layer

In [84]:
def compute_accuracy(x_target, y_target):
    global prediction
    y_predict = sess.run(prediction, feed_dict={xs: x_target, keep_prob:1})
    correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_target, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs:x_target, ys:y_target, keep_prob:1})
    return result

In [57]:
#inputs x & y
with tf.name_scope('inputs'):
    with tf.name_scope('x'):
        xs = tf.placeholder(tf.float32, [None, 784])
        x_image = tf.reshape(xs, [-1, 28, 28, 1])
    with tf.name_scope('y'):
        ys = tf.placeholder(tf.float32, [None, 10])
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32)

In [58]:
#layer1: conv2d->pooling->
with tf.name_scope('layer1_conv'):
    with tf.name_scope('w'):
        W_conv1 = weight_variable([5,5,1,32])
    with tf.name_scope('b'):
        b_conv1 = bias_variable([32])
    with tf.name_scope('conv1'):
        h_conv1 = tf.nn.relu(con2d(x_image, W_conv1) + b_conv1) #28*28*32
    with tf.name_scope('pooling'):
        h_pool1 = pooling(h_conv1) #14*14*32

In [59]:
#layer2: conv2d->pooling->
with tf.name_scope('layer2_conv'):
    with tf.name_scope('w'):
        W_conv2 = weight_variable([5,5,32,64])
    with tf.name_scope('b'):
        b_conv2 = bias_variable([64])
    with tf.name_scope('conv2'):
        h_conv2 = tf.nn.relu(con2d(h_pool1, W_conv2) + b_conv2) #14*14*64
    with tf.name_scope('pooling'):
        h_pool2 = pooling(h_conv2) #7*7*64

In [60]:
#neural network layer1
with tf.name_scope('layer3_nn'):
    with tf.name_scope('reshape'):
        h_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    with tf.name_scope('w'):
        W_nn1 = weight_variable([7*7*64, 1024])
    with tf.name_scope('b'):
        b_nn1 = bias_variable([1024])
    with tf.name_scope('connect'):
        connect = tf.matmul(h_flat, W_nn1) + b_nn1
    h_nn1 = tf.nn.relu(connect)
    h_nn1_dropout = tf.nn.dropout(h_nn1, keep_prob)

In [61]:
#neural network layer2
with tf.name_scope('layer4_nn'):
    with tf.name_scope('w'):
        W_nn2 = weight_variable([1024, 10])
    with tf.name_scope('b'):
        b_nn2 = bias_variable([10])
    with tf.name_scope('connect'):
        connect2 = tf.matmul(h_nn1_dropout, W_nn2) + b_nn2
    prediction = tf.nn.softmax(connect2)

In [62]:
#loss
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=[1]))
    tf.summary.scalar('loss', cross_entropy)

In [63]:
#train
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [64]:
#init
init = tf.global_variables_initializer()

In [85]:
#sess
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs/', sess.graph)
    sess.run(init)
    keep_prob_value = 0.5
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:keep_prob_value})
        if i % 50 == 0:
            accuracy = compute_accuracy(mnist.test.images[:1000], mnist.test.labels[:1000])
            print(accuracy)

0.122
0.755
0.853
0.882
0.914
0.915
0.931
0.945
0.934
0.95
0.955
0.953
0.959
0.959
0.965
0.957
0.968
0.969
0.969
0.973
